In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("data/PS_20174392719_1491204439457_log.csv")
data.head(5)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [4]:
data["nameOrig"].unique()

array(['C1231006815', 'C1666544295', 'C1305486145', ..., 'C1162922333',
       'C1685995037', 'C1280323807'], shape=(6353307,), dtype=object)

In [5]:
data["nameOrig"].value_counts()

nameOrig
C1677795071    3
C1999539787    3
C724452879     3
C1976208114    3
C400299098     3
              ..
C1970706589    1
C40604503      1
C1614818636    1
C2089752665    1
C154988899     1
Name: count, Length: 6353307, dtype: int64

In [6]:
data.shape

(6362620, 11)

In [7]:
data.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [8]:
data.duplicated().sum()

np.int64(0)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [13]:
data["hour"] = pd.to_timedelta(data["step"] , unit = "h")

In [14]:
data["hour"].head()

0   0 days 01:00:00
1   0 days 01:00:00
2   0 days 01:00:00
3   0 days 01:00:00
4   0 days 01:00:00
Name: hour, dtype: timedelta64[ns]

In [17]:
base_time = pd.to_datetime("2017-01-01")

In [18]:
data["hour"] = base_time + data["hour"]
data["hour"].head()

0   2017-01-01 01:00:00
1   2017-01-01 01:00:00
2   2017-01-01 01:00:00
3   2017-01-01 01:00:00
4   2017-01-01 01:00:00
Name: hour, dtype: datetime64[ns]

In [19]:
data["hour"]=pd.to_datetime(data["hour"])

In [20]:
users_data = data[["nameOrig", "nameDest"]]
users_data.head()

,nameOrig,nameDest
0,C1231006815,M1979787155
1,C1666544295,M2044282225
2,C1305486145,C553264065
3,C840083671,C38997010
4,C2048537720,M1230701703


In [ ]:
users_data[""]